In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [3]:
directions = ["left", "right", "back"]
all_coordinates = {}
for direction in directions:
    # 비디오 파일 이름들
    input_files = [f"D:\\Pitch\\sample_videos\\{i}_{direction}.mp4" for i in range(1,21)] # 1부터 20까지의 비디오 경로들

In [4]:
for direction in directions:
    # 비디오 파일 이름들
    input_files = [f"D:\\Pitch\\sample_videos\\{i}_{direction}.mp4" for i in range(1,21)] # 1부터 20까지의 비디오 경로들

    for input_file in input_files:
        if not os.path.exists(input_file):
            print(f"{input_file}이 존재하지 않습니다.")
            continue  # 파일이 존재하지 않으면 다음 파일로 넘어감

        # 비디오 파일의 경우 이것을 사용하세요:
        cap = cv2.VideoCapture(input_file)
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # 어깨 관절 좌표화 코드
        right_elbow_coordinates = [] # 우완투수

        with mp_pose.Pose(
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as pose:

            while cap.isOpened():
                success, image = cap.read()
                if not success:
                    print(f"{input_file}을 처리하는데 실패했습니다.")
                    break

                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = pose.process(image)

                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                image_height, image_width, _ = image.shape

                if results.pose_landmarks:
                    right_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
                    right_elbow_coordinates.append((right_elbow.x, right_elbow.y))

        cap.release()

        all_coordinates[input_file] = right_elbow_coordinates

        # 팔꿈치 좌표 출력
        for i, coord in enumerate(right_elbow_coordinates): # 왼손투수의 경우 left_shoulder_coordinates로 바꿔주세요.
            print(f"Frame {i+1}: x = {coord[0] * image_width}, y = {coord[1] * image_height}")

        # 좌표를 데이터프레임에 저장
        df = pd.DataFrame(right_elbow_coordinates, columns=["x", "y"]) # x_back, x_left, x_right, y_back, y_left, y_right 중 선택

        output_dir = "D:/Pitch/coordinates"
        output_filename = os.path.basename(os.path.splitext(input_file)[0]) + "_elbow_coordinates.csv"
        output_file_path = os.path.join(output_dir, output_filename)

        df.to_csv(output_file_path, index=False)

D:\Pitch\sample_videos\1_left.mp4을 처리하는데 실패했습니다.
Frame 1: x = 858.7310814857483, y = 952.2449111938477
Frame 2: x = 858.717691898346, y = 954.6756362915039
Frame 3: x = 859.1399145126343, y = 955.0657081604004
Frame 4: x = 859.8602485656738, y = 955.3047752380371
Frame 5: x = 859.8489189147949, y = 956.0598564147949
Frame 6: x = 860.0912189483643, y = 956.1932945251465
Frame 7: x = 859.8263883590698, y = 957.1409225463867
Frame 8: x = 859.1572308540344, y = 957.3617935180664
Frame 9: x = 859.1265249252319, y = 957.4015045166016
Frame 10: x = 859.2248225212097, y = 957.3020553588867
Frame 11: x = 859.4239926338196, y = 957.2907257080078
Frame 12: x = 859.5812559127808, y = 957.469310760498
Frame 13: x = 859.6529030799866, y = 957.5716209411621
Frame 14: x = 859.6535468101501, y = 957.6789665222168
Frame 15: x = 859.7077488899231, y = 957.7239990234375
Frame 16: x = 859.7069764137268, y = 957.7749252319336
Frame 17: x = 859.9291920661926, y = 958.0922698974609
Frame 18: x = 860.937659740

: 